# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) 
# Project 4: Web Scraping Job Postings

## Business Case Overview

You're working as a data scientist for a contracting firm that's rapidly expanding. Now that they have their most valuable employee (you!), they need to leverage data to win more contracts. Your firm offers technology and scientific solutions and wants to be competitive in the hiring market. Your principal has two main objectives:

   1. Determine the industry factors that are most important in predicting the salary amounts for these data.
   2. Determine the factors that distinguish job categories and titles from each other. For example, can required skills accurately predict job title?

To limit the scope, your principal has suggested that you *focus on data-related job postings*, e.g. data scientist, data analyst, research scientist, business intelligence, and any others you might think of. You may also want to decrease the scope by *limiting your search to a single region.*

Hint: Aggregators like [Indeed.com](https://www.indeed.com) regularly pool job postings from a variety of markets and industries. 

**Goal:** Scrape your own data from a job aggregation tool like Indeed.com in order to collect the data to best answer these two questions.


# Job scrapping

1. Job are scraped from mycareerfutures.sg
2. Job searched with keyword data
3. The job scrapping code can be found in the folder datajob
4. Totally 4000+ jobs are scrapted
5. To match the scope of the project, kept 1000+ jobs that can be determined as data related job from their job title


In [1]:
import pandas as pd
import numpy as np

In [2]:
# load the job scraped from mycareerfuture.sg
job_post = pd.read_csv('D:\GA\project_submission\project-4\datajob\datajob\datajob_0430_250pages.csv')
job_post.head()

,company_name,job_categories,job_experience,job_level,job_link,job_location,job_requirement,job_role_resp,job_salaries_max,job_salaries_min,job_salaries_type,job_skills,job_title,job_type
0,NTT DATA SINGAPORE PTE. LTD.,"Banking and Finance, Information Technology",NaN,"Manager, Professional",https://www.mycareersfuture.sg/job/project-man...,"KEPPEL TOWERS, 10 HOE CHIANG ROAD 089315","Project management,/Scrum Master/Business Anal...",We are looking for versatile project manager w...,"$9,200","$7,500",Monthly,"Budgets,Business Analysis,Business Development...",Project Manager- Data Security,Contract
1,NTUC ENTERPRISE CO-OPERATIVE LTD,"Education and Training, Information Technology",NaN,Middle Management,https://www.mycareersfuture.sg/job/vice-presid...,NaN,Must be highly technical/strong hardware and s...,"COMPANY DESCRIPTION,NTUC Enterprise is the hol...","$13,000","$8,000",Monthly,"Business Analysis,Business Intelligence,Data A...","Vice President, Enterprise Data Warehouse",Full Time
2,BIOFOURMIS SINGAPORE PTE. LTD.,Engineering,NaN,Senior Executive,https://www.mycareersfuture.sg/job/big-data-en...,"VISION EXCHANGE, 2 VENTURE DRIVE 608526","~,Advanced working SQL knowledge and experienc...",We are looking for a savvy Data Engineer to jo...,"$6,700","$5,200",Monthly,"Agile Methodologies,C#,C++,HTML,Integration,Ja...",Big Data Engineer,Permanent
3,NTUC ENTERPRISE CO-OPERATIVE LTD,Information Technology,NaN,Executive,https://www.mycareersfuture.sg/job/data-govern...,NaN,"Requirements:,Relevant years’ experience in en...","COMPANY DESCRIPTION,NTUC Enterprise is the hol...","$8,000","$5,000",Monthly,"Access,Business Analysis,Business Intelligence...",Data Governance Officer,Full Time
4,CHASSASIA (SINGAPORE) PTE. LTD.,Information Technology,NaN,Professional,https://www.mycareersfuture.sg/job/senior-data...,"HENDERSON BUILDING, 221 HENDERSON ROAD 159557",BS degree in Computer Science or a related tec...,Participated in Projects delivery SDLC - from ...,"$7,500","$6,000",Monthly,"Active Directory,Data Center,Integration,ITIL,...",Senior Data Engineer,Full Time


In [3]:
job_post.shape

(4013, 14)

In [4]:
# checking missing data point
job_post.isnull().sum()

company_name           21
job_categories          3
job_experience       3752
job_level              21
job_link                0
job_location          676
job_requirement       108
job_role_resp           0
job_salaries_max        0
job_salaries_min        0
job_salaries_type       0
job_skills              0
job_title              18
job_type               12
dtype: int64

In [5]:
# remove the data without title, without requirement of experience
job_post.drop(index=job_post[job_post.job_title.isnull()].index, inplace=True)
job_post.drop(index=job_post[job_post.job_level.isnull()].index, inplace=True)

In [6]:
job_post.isnull().sum()

company_name            3
job_categories          0
job_experience       3721
job_level               0
job_link                0
job_location          662
job_requirement        95
job_role_resp           0
job_salaries_max        0
job_salaries_min        0
job_salaries_type       0
job_skills              0
job_title               0
job_type                0
dtype: int64

In [7]:
job_post.job_salaries_min = job_post.job_salaries_min.map(lambda x: int(x[1:].replace(',', '')))
job_post.job_salaries_max = job_post.job_salaries_max.map(lambda x: int(x[1:].replace(',', '')))
job_post['job_salaries_mean'] = (job_post.job_salaries_min + job_post.job_salaries_max)/2
job_post['job_salaries_range'] = (job_post.job_salaries_max - job_post.job_salaries_min)

In [8]:
job_post.drop_duplicates(keep='first', inplace=True)
job_post.reset_index(drop=True, inplace=True)

In [9]:
job_post.shape

(3970, 16)

## Label job title

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import PorterStemmer, SnowballStemmer, WordNetLemmatizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from textblob import TextBlob

In [11]:
title_stop = list(ENGLISH_STOP_WORDS)
title_stop.extend(['3000', '3500', '4000', 'days', 'contract', '1-year', 'orchard', 
                   'west', 'central', 'week', 'year','i2r','star', 'months','workday', 
                   'day', 'shift', 'ot', 'joo', 'koon', 'years','xaxis','workforce'])

In [12]:
# lemmatizer = WordNetLemmatizer()
# stemmer = PorterStemmer()
# # lemmatized_title= job_post.job_title.map(lambda x: ' '.join([stemmer.stem(w) for w in TextBlob(x.lower()).words]))

In [13]:
countVect = CountVectorizer(ngram_range=(2,3), stop_words=title_stop, token_pattern=r'\b[^\d\W_]+\b', min_df=max(int(len(job_post)/300), 5))
countVect.fit(job_post.job_title)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=13,
        ngram_range=(2, 3), preprocessor=None,
        stop_words=['see', 'front', 'out', 'nevertheless', 'never', 'made', 'themselves', 'an', 'thus', 'nobody', 'put', 'full', 'the', 'well', 'top', 'get', 're', 'fire', 'almost', 'neither', 'then', 'we', 'down', 'ever', 'might', 'cant', 'fifteen', 'whom', 'name', 'show', 'beyond', 'whenever', 'nothing', ...r', 'star', 'months', 'workday', 'day', 'shift', 'ot', 'joo', 'koon', 'years', 'xaxis', 'workforce'],
        strip_accents=None, token_pattern='\\b[^\\d\\W_]+\\b',
        tokenizer=None, vocabulary=None)

In [14]:
# take a look of the high popular job key words
pd.DataFrame(countVect.transform(job_post.job_title).todense(), columns=countVect.get_feature_names()).sum(axis=0).sort_values(ascending=False).head(60)

software engineer           96
data scientist              89
data engineer               82
project manager             65
business analyst            63
research fellow             61
data analyst                52
product manager             49
admin assistant             48
senior manager              48
data analytics              48
account manager             48
research engineer           45
big data                    43
accounts executive          41
assistant manager           38
senior software             36
marketing manager           36
t o                         36
customer service            35
research associate          34
technology operations       32
senior engineer             31
network engineer            31
accounts assistant          30
senior executive            28
research assistant          28
senior data                 27
operations executive        27
software developer          26
admin executive             25
vice president              24
senior s

In [15]:
job_post['job_label'] = np.nan

In [16]:
# takindata_entryg all researchers
job_post.loc[job_post.job_title.str.lower().str.contains('research'), 'job_label'] = 'Research'

In [17]:
# take data scientist with the following keywords, if labeled before do not label again
job_post.loc[job_post.job_title.str.lower().str.contains('data scientist') & job_post.job_label.isnull(), 'job_label'] = 'data_scientist'
job_post.loc[job_post.job_title.str.lower().str.contains('data science') & job_post.job_label.isnull(), 'job_label'] = 'data_scientist'
job_post.loc[job_post.job_title.str.lower().str.contains('machine learning') & job_post.job_label.isnull(), 'job_label'] = 'data_scientist'

# data_scientist = pd.concat([data_scientist,job_post[job_post.job_title.str.lower().str.contains('data science')]])
# data_scientist = pd.concat([data_scientist,job_post[job_post.job_title.str.lower().str.contains('machine learning')]])
# data_scientist.drop_duplicates(keep='first', inplace=True)

# # drop if this job is already in research
# data_scientist.drop(index=[i for i in data_scientist.index if i in research.index], axis='index', inplace=True)
# data_scientist.shape

In [18]:
# solution architect 
job_post.loc[job_post.job_title.str.lower().str.contains('architect') & job_post.job_label.isnull(), 'job_label'] = 'solution_architect'
job_post.loc[job_post.job_title.str.lower().str.contains('architecture') & job_post.job_label.isnull(), 'job_label'] = 'solution_architect'
job_post.loc[job_post.job_title.str.lower().str.contains('solution') & job_post.job_label.isnull(), 'job_label'] = 'solution_architect'


In [19]:
# take data enginner with the following keywords
job_post.loc[job_post.job_title.str.lower().str.contains('data engineer') & job_post.job_label.isnull(), 'job_label'] = 'data_engineer'
job_post.loc[job_post.job_title.str.lower().str.contains('data engineering') & job_post.job_label.isnull(), 'job_label'] = 'data_engineer'
job_post.loc[job_post.job_title.str.lower().str.contains('data migration') & job_post.job_label.isnull(), 'job_label'] = 'data_engineer'
job_post.loc[job_post.job_title.str.lower().str.contains('security') & job_post.job_label.isnull(), 'job_label'] = 'data_engineer'
job_post.loc[job_post.job_title.str.lower().str.contains('cloud') & job_post.job_label.isnull(), 'job_label'] = 'data_engineer'
job_post.loc[job_post.job_title.str.lower().str.contains('system engineer') & job_post.job_label.isnull(), 'job_label'] = 'data_engineer'
job_post.loc[job_post.job_title.str.lower().str.contains('network engineer') & job_post.job_label.isnull(), 'job_label'] = 'data_engineer'



# data_engineer = pd.concat([data_engineer,job_post[job_post.job_title.str.lower().str.contains('data engineering')]])
# data_engineer = pd.concat([data_engineer,job_post[job_post.job_title.str.lower().str.contains('data migration')]])
# data_engineer = pd.concat([data_engineer,job_post[job_post.job_title.str.lower().str.contains('security')]])
# data_engineer = pd.concat([data_engineer,job_post[job_post.job_title.str.lower().str.contains('cloud')]])
# data_engineer = pd.concat([data_engineer,job_post[job_post.job_title.str.lower().str.contains('system engineer')]])
# data_engineer = pd.concat([data_engineer,job_post[job_post.job_title.str.lower().str.contains('network engineer')]])

# data_engineer.drop_duplicates(keep='first', inplace=True)
# data_engineer.shape

In [20]:
# take analyst job with the following keywords

job_post.loc[job_post.job_title.str.lower().str.contains('data analytics') & job_post.job_label.isnull(), 'job_label'] = 'data_analyst'
job_post.loc[job_post.job_title.str.lower().str.contains('data analyst') & job_post.job_label.isnull(), 'job_label'] = 'data_analyst'
job_post.loc[job_post.job_title.str.lower().str.contains('data analysis') & job_post.job_label.isnull(), 'job_label'] = 'data_analyst'

# data_analyst = job_post[job_post.job_title.str.lower().str.contains('data analytics')]
# data_analyst = pd.concat([data_analyst,job_post[job_post.job_title.str.lower().str.contains('data analyst')]])
# data_analyst = pd.concat([data_analyst,job_post[job_post.job_title.str.lower().str.contains('data analysis')]])
# data_analyst.drop_duplicates(keep='first', inplace=True)


# data_analyst.shape

In [21]:
# taking all other analysis as other analyst

job_post.loc[job_post.job_title.str.lower().str.contains('analyst') & job_post.job_label.isnull(), 'job_label'] = 'other_analyst'
job_post.loc[job_post.job_title.str.lower().str.contains('analysis') & job_post.job_label.isnull(), 'job_label'] = 'other_analyst'
job_post.loc[job_post.job_title.str.lower().str.contains('analytics') & job_post.job_label.isnull(), 'job_label'] = 'other_analyst'


In [22]:
# taking all other analysis as other analyst
job_post.loc[job_post.job_title.str.lower().str.contains('data entry') & job_post.job_label.isnull(), 'job_label'] = 'entry'


In [23]:
job_post.job_label.value_counts()

other_analyst         288
data_engineer         238
Research              236
solution_architect    123
data_scientist        119
data_analyst           81
entry                  14
Name: job_label, dtype: int64

In [28]:
q1_jobs = job_post[job_post.job_label.notnull()]

### QUESTION 1: Factors that impact salary

To predict salary you will be building either a classification or regression model, using features like the location, title, and summary of the job. If framing this as a regression problem, you will be estimating the listed salary amounts. You may instead choose to frame this as a classification problem, in which case you will create labels from these salaries (high vs. low salary, for example) according to thresholds (such as median salary).

You have learned a variety of new skills and models that may be useful for this problem:
- NLP
- Unsupervised learning and dimensionality reduction techniques (PCA, clustering)
- Ensemble methods and decision tree models
- SVM models

Whatever you decide to use, the most important thing is to justify your choices and interpret your results. *Communication of your process is key.* Note that most listings **DO NOT** come with salary information. You'll need to able to extrapolate or predict the expected salaries for these listings.

In [47]:
q1_jobs.columns

Index(['company_name', 'job_categories', 'job_experience', 'job_level',
       'job_link', 'job_location', 'job_requirement', 'job_role_resp',
       'job_salaries_max', 'job_salaries_min', 'job_salaries_type',
       'job_skills', 'job_title', 'job_type', 'job_salaries_mean',
       'job_salaries_range', 'job_label'],
      dtype='object')

In [49]:
q1_jobs.isnull().sum()

company_name             0
job_categories           0
job_experience        1033
job_level                0
job_link                 0
job_location           198
job_requirement         39
job_role_resp            0
job_salaries_max         0
job_salaries_min         0
job_salaries_type        0
job_skills               0
job_title                0
job_type                 0
job_salaries_mean        0
job_salaries_range       0
job_label                0
dtype: int64

In [ ]:
The choosen features are: job_categories, job_level, job_skills, job_label

In [72]:
q1_jobs.job_type.unique()

array(['Contract', 'Permanent', 'Full Time', 'Permanent, Full Time',
       'Permanent, Contract', 'Contract, Full Time',
       'Contract, Flexi work, Freelance, Full Time, Internship, Part Time, Permanent, Temporary',
       'Contract, Internship', 'Permanent, Contract, Full Time',
       'Full Time, Internship', 'Temporary, Contract, Full Time',
       'Freelance', 'Part Time',
       'Part Time, Permanent, Contract, Full Time', 'Temporary, Full Time',
       'Temporary, Contract', 'Part Time, Contract, Full Time',
       'Part Time, Contract', 'Permanent, Temporary, Contract, Freelance',
       'Temporary'], dtype=object)

In [55]:
q1_jobs.groupby(['job_label']).mean()

,job_salaries_max,job_salaries_min,job_salaries_mean,job_salaries_range
job_label,,,,
Research,12396.699153,7341.906780,9869.302966,5054.792373
data_analyst,8381.728395,5288.135802,6834.932099,3093.592593
data_engineer,9855.962185,6296.974790,8076.468487,3558.987395
data_scientist,17100.336134,10955.126050,14027.731092,6145.210084
entry,2021.428571,1500.000000,1760.714286,521.428571
other_analyst,12890.406250,8260.784722,10575.595486,4629.621528
solution_architect,11122.178862,6872.894309,8997.536585,4249.284553


In [73]:
q1_jobs.groupby(['job_type']).mean()

,job_salaries_max,job_salaries_min,job_salaries_mean,job_salaries_range
job_type,,,,
Contract,9895.466667,6243.016667,8069.241667,3652.450000
"Contract, Flexi work, Freelance, Full Time, Internship, Part Time, Permanent, Temporary",2200.000000,800.000000,1500.000000,1400.000000
"Contract, Full Time",6182.320755,3723.018868,4952.669811,2459.301887
"Contract, Internship",2500.000000,2000.000000,2250.000000,500.000000
Freelance,1500.000000,1000.000000,1250.000000,500.000000
Full Time,12549.843844,7342.363363,9946.103604,5207.480480
"Full Time, Internship",5333.333333,2166.666667,3750.000000,3166.666667
Part Time,1000.000000,800.000000,900.000000,200.000000
"Part Time, Contract",14000.000000,11000.000000,12500.000000,3000.000000


In [56]:
q1_jobs.groupby(['job_level']).mean()

,job_salaries_max,job_salaries_min,job_salaries_mean,job_salaries_range
job_level,,,,
Executive,7924.159624,4527.192488,6225.676056,3396.967136
"Executive, Junior Executive",60000.000000,50000.000000,55000.000000,10000.000000
"Executive, Junior Executive, Senior Executive",4675.000000,3350.000000,4012.500000,1325.000000
"Executive, Non-executive",7100.000000,3550.000000,5325.000000,3550.000000
"Executive, Senior Executive",6208.333333,4183.333333,5195.833333,2025.000000
Fresh/entry level,6730.232558,4238.139535,5484.186047,2492.093023
"Fresh/entry level, Executive",5000.000000,2700.000000,3850.000000,2300.000000
"Fresh/entry level, Executive, Junior Executive, Senior Executive",5400.000000,2700.000000,4050.000000,2700.000000
"Fresh/entry level, Executive, Senior Executive",5800.000000,3500.000000,4650.000000,2300.000000


In [62]:
class MyTokenizer(object):
    def __call__(self,s):
        return s.split(',')


In [63]:
job_level_countVect = CountVectorizer(tokenizer=MyTokenizer(), analyzer='word')

In [67]:
pd.DataFrame(job_level_countVect.fit_transform(q1_jobs.job_level).todense(),columns=job_level_countVect.get_feature_names() )

,executive,fresh/entry level,junior executive,manager,middle management,non-executive,professional,senior executive,executive,fresh/entry level,junior executive,manager,middle management,non-executive,professional,senior executive,senior management
0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
7,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [ ]:
job_level_countVect.fit_transform(q1_jobs.job_level)

In [58]:
q1_jobs.job_level.unique()

array(['Manager, Professional', 'Senior Executive', 'Professional',
       'Manager', 'Senior Management', 'Executive',
       'Professional, Senior Executive', 'Middle Management',
       'Professional, Executive', 'Junior Executive',
       'Professional, Executive, Senior Executive', 'Non-executive',
       'Fresh/entry level',
       'Fresh/entry level, Executive, Senior Executive',
       'Middle Management, Manager', 'Executive, Senior Executive',
       'Fresh/entry level, Junior Executive, Senior Executive',
       'Fresh/entry level, Junior Executive',
       'Middle Management, Manager, Professional',
       'Senior Management, Middle Management, Manager, Senior Executive',
       'Executive, Junior Executive, Senior Executive',
       'Manager, Executive', 'Professional, Executive, Junior Executive',
       'Professional, Executive, Junior Executive, Senior Executive',
       'Fresh/entry level, Professional, Executive, Non-executive, Junior Executive',
       'Fresh/entry l